
<center><img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px"></center>


<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Introduction au Deep Learning avec Keras</h1></center>
<center><h2>Prédictions à l'aide de Dense Neural Network sur des données tabulaires</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

## Contexte et Objectif

>Comme évoqué précedemment, l'algorithme du perceptron simple n'a pas réellement d'intérêt. Il est préférable de considérer un perceptron multicouche pour observer des résultats intéressants. Le GIF suivant en est une illustration : 
>
><img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/masterclass_deeplearning_debutant_intro_dense.gif' style='width:400px'>
>
>**Nous allons à présent nous pencher sur un exemple de modèle de Perceptron multicouche sur le Dataset bien connu IRIS.** Pour rappel, ce jeu de données est le plus réputé pour des projets de reconnaissance d'espèces. Il contient des informations visuelles sur 3 espèces d'iris décrites par 50 observations chacunes. L'objectif est donc de reconnaître la fleur d'iris à partir de certaines mesures caractéristiques regroupées dans des données **tabulaires**.  

* **(a)** Exécuter la cellule suivante pour importer les packages nécéssaires :



In [1]:
import numpy as np # Pour la manipulation de tableaux

import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

from tensorflow.keras.layers import Input, Dense #Pour instancier une couche Dense et une d'Input
from tensorflow.keras.models import Model



## Chargement des données et preprocessing

* **(b)** Lire le fichier **Iris.csv** dans un dataframe df, spécifier la colonne **Id** qui doit contenir les index, et afficher les 5 premières lignes.



In [2]:
## Insérez votre code ici


In [3]:
df=pd.read_csv('Iris.csv', index_col = 'Id')
df.head()


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa



* **(c)** Séparer les variables explicatives de la variable cible.
* **(d)** Encoder la variable cible, **`'Species'`** en fonction de chaque espèce.



In [4]:
## Insérez votre code ici


In [5]:
#Splitting the data into training and test test
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
Y = encoder.fit_transform(y)



* **(e)** Séparer les données en un jeu d'entraînement et test. Le jeu de test pèsera pour **un tiers** du jeu de données.



In [6]:
## Insérez votre code ici


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=42) 



## Multi Layer Perceptron

> Le modèle Perceptron multicouche est une **séquence de couches Perceptron** dont l'entrée est la sortie de la **couche précédente**.
>
> Considérons un modèle Perceptron à 3 couches. Pour un vecteur d'entrée donné $x$, la sortie de la première couche est :
>
> $$ H_1 = \mathrm{Layer_1}(x) $$ 
>
> Ensuite, le vecteur $H_1$ est donné en entrée à la deuxième couche : 
>
> $$ H_2 = \mathrm{Layer_2}(H_1) $$
>
> Enfin, le vecteur $H_2$ est donné en entrée à la troisième couche pour obtenir la sortie finale du modèle :
>
> $$ O = \mathrm{Layer_3}(H_2) $$
>
> <img src = "https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/perceptron3.png">
>
> Les perceptrons multicouches peuvent être construits **sequentiellement** en empilant les couches denses les unes après les autres. Vous rencontrerez parfois cette écriture dans Keras, même si dans ce module nous avons décidé d'utiliser la construction **fonctionnelle** qui est plus versatile et qui s'utilise d'avantage quand on écrit des modèles complexes pouvant avoir une structure non linéaire ou prendre différents inputs.

## Construction et Entraînement d'un modèle




> Nous allons construire notre modèle en ajoutant couche par couche depuis la couche d'entrée jusqu'à la couche de sortie.
>
> La construction d'un modèle avec **Keras** se fait très facilement avec les étapes suivantes:
>
>
>* **Étape 1** : Importer les classes `Input` et `Dense` du sous-module `tensorflow.keras.layers` ainsi que la classe `Model` du sous-module `tensorflow.keras.models`
>
>```python
> from tensorflow.keras... import ...
>```
>
> * **Étape 2** : Instancier une couche d'entrée qui contient les dimensions de nos données en entrée
>
>```python
> inputs = Input(shape = (..), name = "Input")
>```
>
> * **Étape 3** : Instancier les couches qui composeront le modèle avec leur constructeur. Pour instancier une couche dense, il faut utiliser le constructeur `Dense`, que nous avons importé.
>
>```python
> dense1 = Dense(units = ..., activation = "...", name = "Couche_1")
>```
>
> * **Étape 4** : Appliquer les couches une à une (construction **fonctionnelle**).
>
>```python
> x = dense1(inputs)
> x = dense2(x)
> ...
>```
>
> L'instanciation de couches contient plusieurs nuances:
>
> * La première couche que nous allons ajouter au modèle doit être instanciée **en précisant les dimensions du vecteur d'entrée** avec le paramètre **shape**. Cette précision n'est pas nécessaire pour les couches suivantes.
>
>
> * Le **nombre de neurones** dans une couche se définit avec le paramètre **units**.
>
>
> * Pour ajouter une fonction activation à une couche de neurones, on peut soit instancier une couche d'activation puis l'ajouter au modèle, ou bien définir la fonction d'activation dans le paramètre **activation** du constructeur d'une couche.
>

* **(a)** Instancier une couche inputs, avec pour dimension le nombre de variables explicatives du modèle.


* **(b)** Instancier une couche dense appelée **`dense1`** avec 10 neurones. Cette couche aura comme fonction d'activation la fonction `tanh`. 


* **(c)** Instancier une deuxième couche dense appelée **`dense2`** avec 8 neurones. La couche aura comme fonction d'activation `tanh`.


* **(d)** Instancier une troisième couche dense appelée **`dense3`** avec 6 neurones. La couche aura comme fonction d'activation `tanh`.


* **(e)** Instancier une quatrième couche dense appelée **`dense4`** avec 3 neurones. La couche aura comme fonction d'activation `softmax`.



In [8]:
## Insérez votre code ici


In [9]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape = (4), name = "Input")

dense1 = Dense(units = 10, activation = "tanh", name = "Dense_1")
dense2 = Dense(units = 8, activation = "tanh", name = "Dense_2")
dense3 = Dense(units = 6, activation = "tanh", name = "Dense_3")
dense4 = Dense(units = 3, activation = "softmax", name = "Dense_4")



* **(f)** Comme nous sommes dans une construction **fonctionnelle**, il faut appliquer une à une les différentes couches du modèle en précisant que la première couche prendra en entrée l'input et l'output correspondra à l'application de la derniere couche Dense.



In [10]:
## Insérez votre code ici


In [11]:
x=dense1(inputs)
x=dense2(x)
x=dense3(x)
outputs=dense4(x)



> Les commandes suivantes permettent de finaliser la définition du modèle et d'en afficher la structure.



In [12]:
model = Model(inputs = inputs, outputs = outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 4)]               0         
                                                                 
 Dense_1 (Dense)             (None, 10)                50        
                                                                 
 Dense_2 (Dense)             (None, 8)                 88        
                                                                 
 Dense_3 (Dense)             (None, 6)                 54        
                                                                 
 Dense_4 (Dense)             (None, 3)                 21        
                                                                 
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________



* **(g)** Compiler le modèle, avec comme fonction de perte: **`"sparse_categorical_crossentropy"`**, adaptée à la classification à multi-classe. Définir l'optimiseur : **`"adam"`** et la métrique : **`["accuracy"]`**



In [13]:
## Insérez votre code ici


In [14]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])



>Nous pouvons enfin entrainer notre modèle aux données. Ceci est fait dans le même style que *scikit-learn* :
>
>```python
> model.fit (X, y, epochs = n, batch_size = 32, validation_split = p)
>```
>
>* L'argument **`batch_size`** définira le nombre d'échantillons d'apprentissage qui seront utilisés pour calculer le gradient de la fonction de perte. Cela permet de ne pas utiliser toutes les données d'un seul coup, cela a des effets bénéfiques tels que la régularisation et l'accélération du temps d'entraînement.
>
> * L'argument **`epochs`** indique lui le nombre de passages à travers l'ensemble de données que nous allons effectuer pendant le processus d'optimisation. Il y a donc plusieurs étapes d'optimisation à chaque epoch, et chacune de ces étapes utilise un nombre **`batch_size`** de données. Un entraînement avec un **nombre élevé** d'epochs peut induire un **surajustement** ou **surapprentissage** tandis qu'un **faible** nombre d'epochs induit un **sous-ajustement**.
> 
> L'illustration suivante permet de mieux visualiser les arguments epochs et batch_size en fonction de notre jeu de données :
>
><img src=https://datascientest.fr/train/assets/masterclass_deeplearning_batch.png style='width:250px'>
>
> * L'argument **`validation_split`** nous permet de conserver une certaine proportion de l'ensemble de données comme un **ensemble de validation**. Le modèle sera **évalué** sur l'ensemble de données de validation **à la fin de chaque epoch**.
>
> Vous souvenez-vous du jeu de données d'Iris du 1er notebook ? Le widget suivant va illustrer ce concept de batch et epoch à travers la minimisation de la fonction de perte de ce problème. Pour chaque étape d'optimisation, la descente de gradient est effectuée à partir de la loss calculée sur le batch choisi, et cette opération se répète jusqu'à avoir parcouru tout le dataset : c'est à dire jusqu'à avoir effectué une epoch.
* Exécuter la cellule suivante :




In [16]:
%run widget_batch_gradient_descent.py



* **(h)** Entraîner le modèle sur «X_train» et «y_train» pour les epochs **`500`** avec une taille de lot de **`32`** échantillons et en conservant un fractionnement de validation de **`0,1`** .



In [ ]:
## Insérez votre code ici


In [17]:
model.fit(X_train,y_train,epochs=500,batch_size=32,validation_split=0.1)

y_pred = model.predict(X_test)



Epoch 1/500
3/3 [==============================] - 3s 79ms/step - loss: 1.1972 - accuracy: 0.3333 - val_loss: 1.2347 - val_accuracy: 0.4000
Epoch 2/500
3/3 [==============================] - 0s 19ms/step - loss: 1.1768 - accuracy: 0.3333 - val_loss: 1.2154 - val_accuracy: 0.4000
Epoch 3/500
3/3 [==============================] - 0s 19ms/step - loss: 1.1547 - accuracy: 0.3333 - val_loss: 1.1972 - val_accuracy: 0.4000
Epoch 4/500
3/3 [==============================] - 0s 15ms/step - loss: 1.1368 - accuracy: 0.3333 - val_loss: 1.1807 - val_accuracy: 0.4000
Epoch 5/500
3/3 [==============================] - 0s 17ms/step - loss: 1.1227 - accuracy: 0.3333 - val_loss: 1.1652 - val_accuracy: 0.4000
Epoch 6/500
3/3 [==============================] - 0s 17ms/step - loss: 1.1071 - accuracy: 0.3222 - val_loss: 1.1518 - val_accuracy: 0.4000
Epoch 7/500
3/3 [==============================] - 0s 19ms/step - loss: 1.0995 - accuracy: 0.3111 - val_loss: 1.1396 - val_accuracy: 0.4000
Epoch 8/500
3/3 [===


## Performances du modèle

> Nous voulons maintenant effectuer un diagnostic de notre modèle. Pour cela, nous allons calculer une matrice de confusion sur l'échantillon de test.
>
> Néanmoins, si nous essayons de prédire la classe de l'échantillon de test, la méthode **`predict`** du modèle renvoie un **vecteur de probabilités** où chaque élément est la probabilité d'appartenance à la classe correspondant à son indice.
>
> Pour utiliser la fonction `classification_report` du sous-module **metrics** de **scikit-learn**, il faut que le vecteur de la prédiction et le vecteur de la classe réelle soient composés d'entiers.
>
> Nous allons alors utiliser la méthode `argmax` d'un *array* **numpy** pour savoir à quelle classe correspondent les vecteurs binaires et les vecteurs de probabilités.

* **(a)** Prédire les classes de l'échantillon **X_test** à l'aide de la méthode `predict` du modèle. Stocker le résultat dans un tableau nommé **test_pred**.


* **(b)** Appliquer la méthode `argmax` sur les tableaux **test_pred** pour obtenir des vecteurs d'entiers correspondant aux classes prédites et réelles. Il faudra passer l'argument 'axis = 1' pour que l'argmax soit calculée sur les colonnes et non les lignes. Stocker les sorties des appels de la méthode `argmax` dans un tableau nommés **test_pred_class** et les valeurs réelles dans **y_test_class**.




In [18]:
## Insérez votre code ici 


In [19]:
test_pred = model.predict(X_test)


y_test_class = y_test
y_pred_class = np.argmax(test_pred,axis=1)



2/2 [==============================] - 0s 4ms/step



* **(c)** Afficher un compte-rendu évaluatif détaillé de la perfomance du modèle grâce à la fonction `classification_report` du sous-module **metrics** de **scikit-learn**.



In [20]:
## Insérez votre code ici


In [21]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.94      1.00      0.97        15
           2       1.00      0.94      0.97        16

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

[[19  0  0]
 [ 0 15  0]
 [ 0  1 15]]



>Comparativement au modèle avec un seul perceptron, les résultats de classification semble nettemenent meilleurs avec plusieurs couches Denses.

<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center" > Ce qu'il faut retenir </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

> * L'algorithme du **perceptron multicouche** est particulièrement efficace pour les problèmes de **classification**.
> * On utilise la fonction de loss **`sparse categorical crossentropy`** pour un problème de classification. 
> * On peut construire simplement un **MLP** sur **Keras** en utilisant une construction **séquentielle** quand l'architecture est simple. La construction **fonctionnelle** est plus polyvalente. 
> * On entraîne le réseau de neurones en le parcourant à l'aide des paramètres **`batch_size`** et **`epoch`**.
 
>Vous pouvez retrouver toutes les fonctionnalités de **`keras`** dans **`tensorflow.keras`**. Il y a notamment les "couches de neurones" dans **`tensorflow.keras.layers`** :
>
> | Définition          | Classe               |
> | :-----------------: |:--------------------:|
> | Dense layer         | `Dense`              |
> | Convolution 2D      | `Conv2D`             |
> | Dropout             | `Dropout`            |
> | Batch Normalization | `BatchNormalization` |
> | Average Pooling 2D  | `AveragePooling2D`   |
> | Flatten             | `Flatten`            |
> | RNN                 | `RNN`                |
> | LSTM                | `LSTM`               |
> | GRU                 | `GRU`                |

